In [ ]:
from pathlib import Path
import pandas as pd

from plot import create_plots

In [ ]:
pathReal = Path.cwd() / 'smart_meters_london_2013.csv'
pathSynth = ...

df_real = pd.read_csv(pathReal, parse_dates = ['timestamp']).set_index('timestamp')
df_synth = pd.read_csv(pathSynth, parse_dates = ['timestamp']).set_index('timestamp')

In [ ]:
fig_dict, rmse_dict = create_plots(df_real, df_synth)

In [ ]:
score = 0
for item in rmse_dict.values():
    score += item.loc[item['statistic'] != 'median', 'value'].sum()

print(score)